# Basic Information

In [ ]:
import requests
import pandas as pd
# from bs4 import BeautifulSoup
import re
import json
import requests
from scrapy.http import TextResponse
from pprint import pprint
import cloudscraper
import random
import time
# we need headers to disguise our bot as a browser

headers = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36",
    "Accept-Encoding": "gzip,deflate,sdch",
    "Accept-Language": "zh-CN,zh;q=0.8,en-US;q=0.6,en;q=0.4,zh-TW;q=0.2",
}


def scrape_under_cloudflare(url, headers = headers, min_sec = 10):
    print("Request url: {}".format(url))
    
    while True:
        scraper = cloudscraper.create_scraper()
        r = scraper.get(url, headers = headers)
        # load the text to scrapy-type response
        response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

        if 'Cloudflare to restrict access' not in str(response.body.decode()):
            print("\tSucc: {}".format(url))
            break
        else:
            print("\tFail: {}".format(url))
            second = random.randrange(min_sec, min_sec + 1)
            time.sleep(second)
            
    return response


In [ ]:
url = 'https://www.vitals.com/dentists/1qnjkf/Jose_Conde'

response = scrape_under_cloudflare(url, min_sec = 1)

print('Cloudflare to restrict access' in str(response.body.decode()))
# # have a look at its body
# print(str(response.body.decode()))


# with open('_tmp.html', 'w') as f:
#     f.write(str(response.body.decode()))


In [ ]:
# xpath = './/script//text()'
# selectors = response.xpath(xpath)
# js_data = selectors.extract()
# json_str = [i for i in js_data[1].split(';')][0].split('__INITIAL_STATE__=')[-1]
# d = json.loads(json_str)
# pprint(d)

# Json Data

In [ ]:
import json
from pprint import pprint


def process_Json(response):
    xpath = './/script//text()'
    selectors = response.xpath(xpath)
    js_data = selectors.extract()
    json_str = [i for i in js_data[1].split('};')][0].split('__INITIAL_STATE__=')[-1] + '}'
    
    try:
        js_data = json.loads(json_str)
    except:
        js_data = json_str
        print(js_data)
        
    return js_data


js_data = process_Json(response)
[i for i in js_data]

In [ ]:
## all of these are useless.
# for i in [# 'profile',
#          'haserror',
#          'reviewshaserror',
#          'featuredprovidersloaded',
#          'timeslot',
#          'pagedata',
#          'confirm_message_visible',
#          'reviewFilters',
#          'reviewsPageSize',
#          'reviewsDefaultSorting',
#          'recaptchaResponse',
#          'featuredproviders',
#          'ddcTracking',
#          'navigationAndSearch',
#          'route']:
#     print('\n\n'+ i)
#     print(js_data[i])

## Profile

In [ ]:

def process_Vitals(url, min_sec = 10):
    
    print('\n\n=========\nExplore Doctor Profile URL: {}'.format(url))
    response = scrape_under_cloudflare(url, min_sec = min_sec)
    # print('get info from url: {}'.format(url))
    js_data = process_Json(response)
    profile = js_data['profile']

    # fill the data
    profile_data = {}
    for i in [
        # id
        'npi',  'providerid', 'intid', 'entityid_s', 'sponsorid',
        # type
        'display_type', 'profiletype', 'type', 
        # names
        'firstname', 'middlename', 'lastname',  'fullname',  
        # demo
        'gender', 'langspoken',
        # bio
        'about_bio', 'bio_s', 'photourl', 'providerwebsiteurl', 'profileurl','video','quick_facts',
        # practicing,
        'isDentist','is_legacy_provider',  'years_of_experience', 
        'acceptsnewpatients', 'specialty_nimvs', 
        'fallback_specialties', 'fallback_specialty', 'provider_expertise', 
        # awards and certification
        'degreeabbr', 'certifications', 'awards', 'total_awards',
        # education
         'education_nimvs', 
        # hosptial
        'location_nimvs', 'hospital_nimvs','displayhospitals',
        'city', 'state', 'postalcode', 'geolocation', 
        # insurance
        'insurance_facet',  
        # rating and review
        'number_of_ratings', 
        'ratings_per_star', 'rating_score','rating_overview',
        'review_count', 'review_questions', 'recent_reviews',]:

        # print('\n\n'+ i)
        # pprint(profile.get(i, None))
        profile_data[i] = profile.get(i, None)
        
        
    if profile_data['number_of_ratings'] > len(profile_data['recent_reviews']): 
        # keep ascending
        number_of_ratings = profile_data['number_of_ratings'] 
        review_urls = ['{}/reviews?page={}&sort=updated_at_dt%20asc'.format(url, page) 
                       for page in range(1, int(number_of_ratings/12) + 2)]
        profile_data['review_urls'] = review_urls
        
        all_reviews = []
        success_review_urls = []
        for page_idx, review_url in enumerate(review_urls):
            print('\n* Work at page {}'.format(page_idx))
            response = scrape_under_cloudflare(review_url, min_sec = min_sec)
            
            js_data = process_Json(response)
            # print([i for i in js_data])
            reviews = js_data['profile']['reviews']
                
            # all or nothing
            # try:
            #     js_data = process_Json(response)
            #     # print([i for i in js_data])
            #     reviews = js_data['profile']['reviews']
            # except:
            #     print('Failure to parse reviews json from URL: {}'.format(review_url))
            #     continue

            all_reviews = all_reviews + reviews
            print('review number: {}'.format(len(reviews)))
            success_review_urls.append(review_url)
            
        profile_data['review_urls'] = review_urls
        profile_data['success_review_urls'] = success_review_urls
        profile_data['reviews'] = all_reviews
    else:
        profile_data['review_urls'] = []
        profile_data['success_review_urls'] = []
        profile_data['reviews'] = profile_data['recent_reviews']
        
        
    # profile_data['url'] = url
    assert len(profile_data['review_urls']) == len(profile_data['success_review_urls'])
    assert len(profile_data['reviews']) == profile_data['number_of_ratings']
        
        
        
    d = profile_data
    print('\nFor physician {}, get reviews {} vs {}\n======'.format(d['fullname'], len(d['reviews']), d['number_of_ratings']))
    return profile_data




In [ ]:
min_sec = 1
url = 'https://www.vitals.com/dentists/1qnjkf/Jose_Conde'
profile_data = process_Vitals(url, min_sec)

In [ ]:
print([i for i in profile_data])



cols = ['npi', 'providerid', 'intid', 'entityid_s', 'sponsorid', 'display_type', 'profiletype', 'type', 
 'firstname', 'middlename', 'lastname', 'fullname', 'gender', 'langspoken', 'about_bio', 'bio_s',
 'photourl', 'providerwebsiteurl', 'profileurl', 'video', 'quick_facts', 
 'isDentist', 'is_legacy_provider', 'years_of_experience', 'acceptsnewpatients', 'specialty_nimvs', 
 'fallback_specialties', 'fallback_specialty', 'provider_expertise', 'degreeabbr', 
 'certifications', 'awards', 'total_awards', 'education_nimvs', 'location_nimvs', 
 'hospital_nimvs', 'displayhospitals', 'city', 'state', 'postalcode', 'geolocation',
 'insurance_facet', 'number_of_ratings', 'ratings_per_star', 'rating_score', 'rating_overview', 
 'review_count', 'review_questions', 'recent_reviews', 'review_urls', 'success_review_urls', 'reviews', 'url']

# All In One

In [ ]:
url_list = ['https://www.vitals.com/doctors/Dr_Lisa_Kairis.html',
 'https://www.vitals.com/doctors/Dr_Sophie_Hoptowit.html',
 'https://www.vitals.com/doctors/Dr_Stephen_Kelminson.html',
 'https://www.vitals.com/doctors/Dr_Jonathan_W_Weeks.html',
 'https://www.vitals.com/doctors/Dr_John_Condojani.html',
 'https://www.vitals.com/doctors/Dr_Anthony_Leazzo.html',
 'https://www.vitals.com/doctors/Dr_James_Gleason.html',
 'https://www.vitals.com/doctors/Dr_Katherine_Francis.html',
 'https://www.vitals.com/doctors/Dr_Brett_Sanders.html',
 'https://www.vitals.com/doctors/Dr_Choon_PNg.html']


min_sec = 1
for url in url_list:
    # url = 'https://www.vitals.com/dentists/1qnjkf/Jose_Conde'
    profile_data = process_Vitals(url, min_sec)


